<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0c7455841f756da6b0300f18969cb5f432134c8a0e141782bf72768bda06e7f0
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-11 15:10:57
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: 67.15 K (0.48%)
Current PnL: -28.65 L (-18.51%)
CY Booked + Current PnL: -14.36 L (-9.27%)
-------------------
Total profit:  1.27 L
Total loss:  -29.92 L
-------------------
Total Booked + Current PnL: 12.62 L (9.88%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.23%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.28 L (66.8%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,1.03,-16.55,19.94,0.09,16770.0,-16675.0,84102.0,95.30,44.0,M-SC,2.37,253.0,-0.99,0.62,10.49,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.42,-12.50,14.31,0.02,21856.0,-21828.0,152731.0,132.70,52.0,M-SC,16.23,234.0,-1.00,1.13,32.85,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.17,-2.05,24.60,22.05,23609.0,-2010.0,95970.0,-15.54,56.0,H-SC,7.10,164.0,-0.09,0.71,39.50,XR,ATH,FINANCE
43,ITC,452.00,-0.04,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,47.0,X-LC,1.75,5.0,-0.13,1.46,3.94,X40,NTT,FMCG
82,VOLTAS,1530.00,1.03,5.98,12.94,19.69,26295.0,11463.0,203205.0,-4.39,49.0,X-MC,2.57,78.0,0.44,1.51,13.18,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,5.30,-16.48,45.57,21.58,53337.0,-23094.0,117045.0,-57.91,30.0,X-MC,7.07,56.0,-0.43,0.87,5.30,X40N,ATH,IT
72,TATAELXSI,9161.00,3.17,-27.70,82.60,32.03,78737.0,-36512.0,95323.0,-21.75,40.0,H-SC,8.57,148.0,-0.46,0.71,6.77,OX40N,NTT,IT
6,ASIANTILES,137.00,2.79,-15.49,114.97,81.67,91148.0,-14530.0,79280.0,7127.78,48.0,L-SC,14.00,271.0,-0.16,0.59,53.53,XR,NTT,CERAMICS
10,BANDHANBNK,400.00,2.75,-30.14,175.43,92.42,341401.0,-83951.0,194608.0,37.21,43.0,H-SC,8.57,170.0,-0.25,1.44,12.47,XY24,NTT,BANKS
48,KPIGREEN,730.66,2.63,-18.80,80.97,46.95,82383.0,-23552.0,101745.0,-24.83,33.0,H-SC,6.35,154.0,-0.29,0.76,27.46,MH,ATH,POWER


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,CAMPUS,393.0,-1.86,-10.60,49.09,33.28,73629.0,-17787.0,149988.0,-27.96,40.0,M-SC,2.80,222.0,-0.24,1.11,16.25,XY24,NTT,FOOTWEAR
12,BERGEPAINT,680.0,-1.77,-3.91,26.07,21.14,56952.0,-8882.0,218457.0,-10.72,39.0,X-MC,4.18,74.0,-0.16,1.62,23.13,XY24,NTT,PAINTS
15,CAMS,4762.0,-1.64,-80.68,545.96,24.79,265637.0,-203201.0,48655.0,-81.54,36.0,X-SC,1.48,86.0,-0.76,0.36,18.94,X40N,NTT,MISC
16,CERA,9475.0,-1.56,-31.07,79.72,23.89,96665.0,-54647.0,121256.0,-33.41,31.0,H-SC,9.74,157.0,-0.57,0.90,4.72,OX40N,NTT,CERAMICS
30,HATHWAY,31.4,-1.40,-30.13,148.03,73.29,112444.0,-32760.0,75960.0,780.43,39.0,H-SC,7.50,172.0,-0.29,0.56,2.43,XR,NTT,ENTERTAINMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.74,-6.89,123.29,107.91,169490.0,-10171.0,137473.0,-24.20,39.0,M-SC,12.36,216.0,-0.06,1.02,2.50,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.42,-12.50,14.31,0.02,21856.0,-21828.0,152731.0,132.70,52.0,M-SC,16.23,234.0,-1.00,1.13,32.85,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,1.03,-16.55,19.94,0.09,16770.0,-16675.0,84102.0,95.30,44.0,M-SC,2.37,253.0,-0.99,0.62,10.49,OX40N,NTT,DURABLES
66,SIS,528.00,-0.66,-25.37,62.49,21.26,51780.0,-28170.0,82862.0,1953.37,43.0,H-SC,4.61,166.0,-0.54,0.62,11.96,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.51,-25.42,52.15,13.47,104886.0,-68544.0,201123.0,-69.28,31.0,H-SC,4.18,158.0,-0.65,1.49,2.27,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.51,6.35,61.97,72.27,111679.0,10766.0,180214.0,-8.8,67.0,M-LC,1.93,99.0,0.1,1.34,12.92,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.51,6.35,61.97,72.27,111679.0,10766.0,180214.0,-8.80,67.0,M-LC,1.93,99.0,0.10,1.34,12.92,XR,NTT,IT
38,INDIAMART,4810.62,0.21,-3.77,114.82,106.72,136278.0,-4648.0,118688.0,-52.34,32.0,H-SC,1.99,139.0,-0.03,0.88,18.73,AR,ATH,MISC
37,IEX,219.00,2.11,-3.35,53.78,48.63,103930.0,-6704.0,193250.0,-35.77,51.0,H-SC,14.05,136.0,-0.06,1.43,7.81,XR,NTT,MISC
25,FINCABLES,1641.55,-0.74,-6.89,123.29,107.91,169490.0,-10171.0,137473.0,-24.20,39.0,M-SC,12.36,216.0,-0.06,1.02,2.50,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.23,-2.83,39.58,35.63,80601.0,-5934.0,203640.0,-16.74,37.0,H-MC,3.16,119.0,-0.07,1.51,13.37,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.45,-21.64,89.46,48.45,140552.0,-43394.0,157112.0,-27.18,25.0,X-MC,12.19,64.0,-0.31,1.17,2.30,X40N,ATH,FINANCE
33,HINDUNILVR,2922.00,0.11,-10.36,26.81,13.67,61778.0,-26633.0,230430.0,-17.11,29.0,X-LC,2.63,14.0,-0.43,1.71,8.78,XY25,NTT,FMCG
75,TMPV,600.00,0.90,-17.89,73.16,42.18,166042.0,-49452.0,226958.0,-26.74,29.0,X-LC,6.05,3.0,-0.30,1.69,0.90,XY24,NTT,AUTO
22,DIXON,18931.72,5.30,-16.48,45.57,21.58,53337.0,-23094.0,117045.0,-57.91,30.0,X-MC,7.07,56.0,-0.43,0.87,5.30,X40N,ATH,IT
51,MEDANTA,1486.00,-1.05,-4.41,29.80,24.08,36162.0,-5601.0,121349.0,-12.23,31.0,X-SC,5.42,89.0,-0.15,0.90,14.34,XY24,NTT,HEALTHCARE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.0,-1.64,-80.68,545.96,24.79,265637.0,-203201.0,48655.0,-81.54,36.0,X-SC,1.48,86.0,-0.76,0.36,18.94,X40N,NTT,MISC
8,AWL,485.0,-0.00,-21.78,96.24,53.50,227153.0,-65723.0,236028.0,-62.10,33.0,X-MC,1.71,58.0,-0.29,1.75,4.84,XY24,NTT,FMCG
43,ITC,452.0,-0.04,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,47.0,X-LC,1.75,5.0,-0.13,1.46,3.94,X40,NTT,FMCG
36,ICICIPRULI,790.0,-1.07,5.44,24.21,30.97,45115.0,9610.0,186348.0,-19.31,60.0,X-MC,2.02,75.0,0.21,1.38,18.68,X40,ATH,INSURANCE
3,ACC,3906.0,-0.17,-25.22,119.52,64.15,212662.0,-60021.0,177930.0,-56.31,34.0,X-SC,2.10,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.17,-25.22,119.52,64.15,212662.0,-60021.0,177930.0,-56.31,34.0,X-SC,2.10,82.0,-0.28,1.32,0.00,XY24,BTT,CEMENT
54,PGHH,17907.65,-0.26,-6.13,42.50,33.76,80115.0,-12315.0,188505.0,-33.72,33.0,X-MC,5.24,57.0,-0.15,1.40,0.00,X40,ATH,FMCG
31,HAVELLS,2069.16,0.86,-10.24,47.60,32.48,140134.0,-33592.0,294399.0,-14.76,39.0,X-MC,5.49,67.0,-0.24,2.19,0.86,X40,ATH,ELECTRICAL
75,TMPV,600.00,0.90,-17.89,73.16,42.18,166042.0,-49452.0,226958.0,-26.74,29.0,X-LC,6.05,3.0,-0.30,1.69,0.90,XY24,NTT,AUTO
53,PAGEIND,51605.07,1.10,-9.37,39.10,26.07,58024.0,-15340.0,148400.0,-33.27,33.0,X-MC,12.13,55.0,-0.26,1.10,1.10,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-0.02,-29.83,103.56,42.84,47240.0,-19390.0,45616.0,-54.27,41.0,X-SC,36.53,83.0,-0.41,0.34,3.20,XY24,NTT,MISC
15,CAMS,4762.00,-1.64,-80.68,545.96,24.79,265637.0,-203201.0,48655.0,-81.54,36.0,X-SC,1.48,86.0,-0.76,0.36,18.94,X40N,NTT,MISC
59,RELAXO,1176.00,-1.19,-47.31,189.09,52.31,144612.0,-68682.0,76478.0,-43.96,43.0,X-SC,6.42,91.0,-0.47,0.57,2.37,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.29,-38.07,118.33,35.20,94285.0,-48990.0,79680.0,4.40,34.0,X-SC,15.68,92.0,-0.52,0.59,1.21,X40,NTT,FOOTWEAR
22,DIXON,18931.72,5.30,-16.48,45.57,21.58,53337.0,-23094.0,117045.0,-57.91,30.0,X-MC,7.07,56.0,-0.43,0.87,5.30,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.13,-13.23,37.47,19.28,112481.0,-45767.0,300189.0,-24.27,60.0,X-LC,5.21,1.0,-0.41,2.23,10.98,X40,ATH,IT
41,INFY,2275.00,0.83,8.78,42.37,54.87,144216.0,27486.0,340374.0,-15.29,62.0,X-LC,3.23,2.0,0.19,2.53,17.79,X40,BTT,IT
75,TMPV,600.00,0.90,-17.89,73.16,42.18,166042.0,-49452.0,226958.0,-26.74,29.0,X-LC,6.05,3.0,-0.30,1.69,0.90,XY24,NTT,AUTO
81,VBL,671.64,1.10,-3.41,40.47,35.67,123458.0,-10782.0,305060.0,-14.83,57.0,X-LC,4.58,4.0,-0.09,2.27,9.97,X40N,ATH,FMCG
43,ITC,452.00,-0.04,-1.53,12.16,10.44,23963.0,-3071.0,197067.0,-41.15,47.0,X-LC,1.75,5.0,-0.13,1.46,3.94,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.79,-15.49,114.97,81.67,91148.0,-14530.0,79280.0,7127.78,48.0,L-SC,14.00,271.0,-0.16,0.59,53.53,XR,NTT,CERAMICS
50,MASFIN,398.61,0.17,-2.05,24.60,22.05,23609.0,-2010.0,95970.0,-15.54,56.0,H-SC,7.10,164.0,-0.09,0.71,39.50,XR,ATH,FINANCE
13,BSOFT,831.70,1.22,-22.36,94.21,50.79,102881.0,-31449.0,109204.0,-0.27,67.0,H-SC,5.86,151.0,-0.31,0.81,27.15,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.42,-12.50,14.31,0.02,21856.0,-21828.0,152731.0,132.70,52.0,M-SC,16.23,234.0,-1.00,1.13,32.85,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.92,-2.88,24.53,20.95,59980.0,-7251.0,244517.0,-3.87,47.0,X-LC,18.65,31.0,-0.12,1.82,31.73,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.22,-22.36,94.21,50.79,102881.0,-31449.0,109204.0,-0.27,67.0,H-SC,5.86,151.0,-0.31,0.81,27.15,XR,ATH,IT
56,RAJESHEXPO,518.00,1.20,-62.81,169.08,0.07,86256.0,-86162.0,51015.0,1769.95,58.0,L-SC,2.39,268.0,-1.00,0.38,26.88,OX40N,NTT,JEWELLERY
84,WIPRO,420.00,0.51,6.35,61.97,72.27,111679.0,10766.0,180214.0,-8.80,67.0,M-LC,1.93,99.0,0.10,1.34,12.92,XR,NTT,IT
32,HCLTECH,1922.01,0.37,8.52,14.94,24.73,39223.0,20611.0,262535.0,13.18,64.0,X-LC,6.22,7.0,0.53,1.95,25.05,X40,ATH,IT
0,5PAISA,593.00,1.88,-41.20,92.00,12.90,118203.0,-90014.0,128482.0,88.70,44.0,H-SC,11.64,161.0,-0.76,0.95,6.83,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.78
1,25,45.75
2,50,76.67


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.28
LC    24.67
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.28
X40      22.43
X40N     10.85
XR       10.13
XY25      9.39
AR        9.35
OX40N     7.78
SR        1.02
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.56
X-MC    23.52
X-LC    21.10
M-SC    12.11
X-SC     6.93
H-MC     5.00
L-SC     1.44
M-MC     1.41
M-LC     1.34
H-LC     1.18
L-LC     1.05
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.37,-7.55,42.33
IT,12.79,-17.03,80.30
FINANCE,9.60,-19.99,70.69
ELECTRICAL,6.25,-11.67,52.57
MISC,6.09,-58.26,123.10
PAINTS,6.06,-13.04,29.83
INSURANCE,4.60,-2.09,38.07
PHARMA,4.11,-0.90,33.19
AUTO,3.42,-25.17,78.84


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3227658.0,21
AR,1311694.0,10
XR,1295602.0,13
X40,1018102.0,14
X40N,989262.0,9
OX40N,743040.0,10
XY25,377497.0,6
SR,282689.0,2
MH,82383.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3667642.0,25
M-SC,1426414.0,15
X-MC,1316593.0,16
X-SC,986281.0,8
X-LC,923361.0,11
H-MC,405154.0,3
L-SC,267163.0,3
M-LC,111679.0,1
H-LC,74585.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1283312.0      6
           AR         899601.0      5
M-SC       XY24       813614.0      6
H-SC       XR         774272.0      7
X-SC       X40N       508123.0      3
X-MC       X40        456145.0      7
           XY24       397099.0      3
X-LC       X40        379863.0      5
H-SC       OX40N      345385.0      4
M-SC       OX40N      311399.0      5
X-SC       XY24       296064.0      3
H-SC       SR         282689.0      2
X-MC       X40N       279691.0      4
X-LC       XY24       242272.0      2
H-MC       AR         209857.0      2
X-LC       X40N       201448.0      2
H-MC       XY24       195297.0      1
X-MC       XY25       183658.0      2
X-SC       X40        182094.0      2
L-SC       XR         180907.0      2
M-SC       XR         173750.0      2
           AR         127651.0      2
M-LC       XR         111679.0      1
X-LC       XY25        99778.0      2
L-SC       OX40N       86256.0      1
H-SC       MH          82383.0      1
H-LC       AR          74585.0      1
M-MC       XY25        56068.0      1
L-MC       XR          54994.0      1
L-LC       XY25        37993.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
